# Result Cleaning and Evaluation for Binary Cognitive Distortion Detection

In [18]:
import os
import pandas as pd
import re
import csv

In [19]:
csv.field_size_limit(1000) 
eval_path = '/Users/ulugsali/Desktop/Cognitive-Distortion-Project/processed_dataset.csv'
inference_df = pd.read_csv(eval_path)

pd.set_option('display.max_colwidth', None)
inference_df.head()

,Unnamed: 0,Prompt,Response,Class_1,Class_2
0,0,"My husband works a lot which really helps our family. The problem is he never wants to go anywhere with my children and me. When he gets home from work, he’s highly irritable and often goes around the house criticizing me for practically everything. Cleaning, watching the kids, what I did or didn’t do during the day, Ext. Most every time I talk with him, he has something negative to say about me. Something I didn’t do right. This is really starting to wear on me. My Self Esteem is pretty bad and this makes it worse. I also get very anxious when I know he’s coming home soon, because I know it is going to be constant critiques. Also, when I ask him if he wants to go someplace with me and the kids, he never wants to and gets upset about it. I get in a bad mood too and he ends up leaving. In which doesn’t make any sense because he said he didn’t want to go anywhere. I have told him how much these things bother me. Every time, he turns around and says what bother him. Nothing gets resolved. I work from home and it’s difficult because I am also watching the kids. He doesn’t like anyone else watching the kids so I try to the majority of the time, unless I have somewhere important to go to. We used to be a lot different. We would go out to dinner, shopping, ext. We would watch TV together and he really didn’t critique me all too much. He used to say good things about me in which he doesn’t say anymore. For example, he used to call me “pretty girl”. He has not said that for awhile now. We have also went through some crap that hurt our relationship. I wasn’t very mature when we started dating so I made some poor decisions that I would never do presently. I’d love to know what changed our relationship so much? Is he acting like this for revenge or because he’s not happy? I can honestly say I’m miserable when I’m around him. I cannot say I’m overall miserable because I do enjoy spending time with my children. My underneath layer is hurt, badly. I feel like my feelings don’t matter. I don’t know when and if to call it quits. We have been married for 6 years and I don’t give up easily. I do love him but I wish he was how he was when we got married. He actually cared and SHOWED me how much he loved me.","Let's start eliminating the least likely classes present in the anecdote:\n\n1. Fortune-telling: This class is unlikely because there is no indication that the patient expects a specific outcome or believes that an event will always result in the worst possible outcome.\n\n2. Mind Reading: While the patient does mention their husband's behavior, there is no evidence to suggest that they believe he is intentionally trying to make them feel bad or react negatively without basis.\n\n3. Personalization: The patient does take responsibility for some of their actions and decisions, but there is also a significant amount of blame placed on their husband's behavior, making this class less likely.\n\n4. Labeling: There is no explicit labeling of themselves or their husband as ""loser"" or any other negative label, making this class less likely.\n\n5. Should Statements: While the patient does express some expectations about what they want from their relationship (e.g., wanting him to be more like he used to be), these statements are not overly rigid or absolute, making this class less likely.\n\n6. All-or-Nothing Thinking: The patient's descriptions of their husband's behavior do not suggest a binary thinking pattern where anything short of perfection is seen as a failure.\n\n7. Emotional Reasoning: While the patient does express strong emotions and frustration, there is no indication that they believe their feelings are objective truth or evidence of reality.\n\n8. Magnification: The patient does exaggerate some aspects of their husband's behavior (e.g., saying he constantly critiques them), but this class seems less likely because it doesn't capture the full scope of their concerns.\n\n9. Overgenerali

Fix Common Variations

In [30]:
def remove_punctuation(text):
    if pd.isna(text):
        return 'nan'
    text = re.sub(r'[^\w\s]', '', text)
    return re.sub(r'\n', '', text)

# Lowercase and clean Class_1
inference_df['Class_1'] = inference_df['Class_1'].apply(lambda x: x.lower() if pd.notna(x) else 'nan')
inference_df['Class_1'] = inference_df['Class_1'].apply(remove_punctuation)

# Lowercase and clean Class_2
inference_df['Class_2'] = inference_df['Class_2'].apply(lambda x: x.lower() if pd.notna(x) else 'nan')
inference_df['Class_2'] = inference_df['Class_2'].apply(remove_punctuation)

# If you still want to process Response column as well
inference_df['Response'] = inference_df['Response'].apply(lambda x: x.lower() if pd.notna(x) else 'nan')
inference_df['Response'] = inference_df['Response'].apply(remove_punctuation)

# Display the results
inference_df.head()

,Unnamed: 0,Prompt,Response,Class_1,Class_2
0,0,"My husband works a lot which really helps our family. The problem is he never wants to go anywhere with my children and me. When he gets home from work, he’s highly irritable and often goes around the house criticizing me for practically everything. Cleaning, watching the kids, what I did or didn’t do during the day, Ext. Most every time I talk with him, he has something negative to say about me. Something I didn’t do right. This is really starting to wear on me. My Self Esteem is pretty bad and this makes it worse. I also get very anxious when I know he’s coming home soon, because I know it is going to be constant critiques. Also, when I ask him if he wants to go someplace with me and the kids, he never wants to and gets upset about it. I get in a bad mood too and he ends up leaving. In which doesn’t make any sense because he said he didn’t want to go anywhere. I have told him how much these things bother me. Every time, he turns around and says what bother him. Nothing gets resolved. I work from home and it’s difficult because I am also watching the kids. He doesn’t like anyone else watching the kids so I try to the majority of the time, unless I have somewhere important to go to. We used to be a lot different. We would go out to dinner, shopping, ext. We would watch TV together and he really didn’t critique me all too much. He used to say good things about me in which he doesn’t say anymore. For example, he used to call me “pretty girl”. He has not said that for awhile now. We have also went through some crap that hurt our relationship. I wasn’t very mature when we started dating so I made some poor decisions that I would never do presently. I’d love to know what changed our relationship so much? Is he acting like this for revenge or because he’s not happy? I can honestly say I’m miserable when I’m around him. I cannot say I’m overall miserable because I do enjoy spending time with my children. My underneath layer is hurt, badly. I feel like my feelings don’t matter. I don’t know when and if to call it quits. We have been married for 6 years and I don’t give up easily. I do love him but I wish he was how he was when we got married. He actually cared and SHOWED me how much he loved me.",lets start eliminating the least likely classes present in the anecdote1 fortunetelling this class is unlikely because there is no indication that the patient expects a specific outcome or believes that an event will always result in the worst possible outcome2 mind reading while the patient does mention their husbands behavior there is no evidence to suggest that they believe he is intentionally trying to make them feel bad or react negatively without basis3 personalization the patient does take responsibility for some of their actions and decisions but there is also a significant amount of blame placed on their husbands behavior making this class less likely4 labeling there is no explicit labeling of themselves or their husband as loser or any other negative label making this class less likely5 should statements while the patient does express some expectations about what they want from their relationship eg wanting him to be more like he used to be these statements are not overly rigid or absolute making this class less likely6 allornothing thinking the patients descriptions of their husbands behavior do not suggest a binary thinking pattern where anything short of perfection is seen as a failure7 emotional reasoning while the patient does express strong emotions and frustration there is no indication that they believe their feelings are objective truth or evidence of reality8 magnification the patient does exaggerate some aspects of their husbands behavior eg saying he constantly critiques them but this class seems less likely because it doesnt capture the full scope of their concerns9 overgeneralization the patient mentions that their husband used to be more positive and affectionate but th

Create Mapping Function

In [34]:
distortions = {
    'none': 0,
    'no distortion': 0,
    'emotional reasoning': 1,
    'overgeneralization': 2,
    'mental filter': 3,
    'should statements': 4,
    'all-or-nothing thinking': 5,
    'mind reading': 6,
    'cognitive distortion: fortune-telling': 7,
    'magnification': 8,
    'minimization': 8,
    'magnification/minimization': 8,
    'personalization': 9,
    'labeling': 10,
}

patterns = {
    'none': r'\bnone\b',
    'no distortion': r'\bno distortion\b',
    'no distortion': r'no distortion',
    'emotional reasoning': r'\bemotional reasoning\b',
    'overgeneralization': r'\bovergeneralization\b',
    'mental filter': r'\bmental filter\b',
    'should statements': r'\bshould statements\b',
    'all-or-nothing thinking': r'\ball-or-nothing thinking\b',
    'all-or-nothing thinking': r'\ball or nothing thinking\b',
    'mind reading': r'\bmind reading\b',
    'magnification/minimization': r'\bmagnification/minimization\b',
    'magnification': r'\bmagnification\b',
    'minimization': r'\bminimization\b',
    'personalization': r'\bpersonalization\b',
    'labeling': r'\blabeling\b',
}

def catch_responses(x):
    """
    Match text against patterns to identify distortions.
    Returns the corresponding distortion code.
    """
    if pd.isna(x) or x == 'nan':
        return -1
        
    normalized_text = x.lower()
    for label, pattern in patterns.items():
        match = re.search(pattern, normalized_text)
        if match:
            return distortions[label]
    return -1

# Create new columns for the numeric labels
inference_df['Class_1_Label'] = inference_df['Class_1'].apply(catch_responses)
inference_df['Class_2_Label'] = inference_df['Class_2'].apply(catch_responses)

# Display the first few rows to verify the labeling
inference_df[['Class_1', 'Class_1_Label', 'Class_2', 'Class_2_Label']].head(50)

,Class_1,Class_1_Label,Class_2,Class_2_Label
0,mental filter,3,allornothing thinking,-1
1,nan,-1,nan,-1
2,personalization,9,nan,-1
3,mental filter,3,nan,-1
4,labeling,10,nan,-1
5,mind reading,6,nan,-1
6,nan,-1,nan,-1
7,overgeneralization,2,nan,-1
8,magnification,8,nan,-1
9,nan,-1,nan,-1


Gather Gold Data

In [36]:
gold_dominant_data = pd.read_csv('../../datasets/test.csv')['Dominant Distortion']\
    .apply(lambda x: x.lower())\
    .apply(catch_responses)
gold_secondary_data = pd.read_csv('~/Desktop/Cognitive-Distortion-Project/datasets/test.csv')['Secondary Distortion (Optional)']\
    .apply(lambda x: x if x is None else str(x).lower())\
    .apply(catch_responses)

inference_df['gold_dominant'] = gold_dominant_data
inference_df['gold_secondary'] = gold_secondary_data

In [37]:
def match_either(row):
    if row['Response'] == -1:
        row['gold'] = row['gold_dominant']
        return row
    if row['Response'] == row['gold_secondary']:
        row['gold'] = row['gold_secondary']
    else:
        row['gold'] = row['gold_dominant']
    return row

In [40]:
inference_df = inference_df.apply(match_either, axis=1)

In [43]:
print("Unique classes in 'gold':", inference_df['gold'].unique())
print("Unique classes in 'Class_1_Label':", inference_df['Class_1_Label'].unique())
print("Unique classes in 'Class_2_Label':", inference_df['Class_2_Label'].unique())
#num in each class
print(inference_df['gold'].value_counts())

Unique classes in 'gold': [ 2  0  3  6  1 10  4 -1  9  8]
Unique classes in 'Class_1_Label': [ 3 -1  9 10  6  2  8  1  0  4]
Unique classes in 'Class_2_Label': [-1  1  3 10]
gold
 0     163
 2      60
-1      55
 6      51
 10     42
 8      37
 3      27
 9      26
 1      24
 4      21
Name: count, dtype: int64


Compute Accuracy and F1

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, classification_report
import numpy as np

def evaluate_predictions(df):
    """
    Evaluate predictions against gold column.
    Calculates overall metrics and per-class F1.
    Completely ignores -1 values in both predictions and gold labels.
    """
    # Filter out rows where both predictions are NaN (-1)
    valid_rows = ~((df['Class_1_Label'] == -1) & (df['Class_2_Label'] == -1))
    df_valid = df[valid_rows].copy()
    
    # Create arrays for true and predicted values
    y_true = []
    y_pred = []
    
    for _, row in df_valid.iterrows():
        # Skip if gold label is -1
        if row['gold'] == -1:
            continue
            
        # Predicted labels (excluding -1)
        pred_labels = [label for label in [row['Class_1_Label'], row['Class_2_Label']] 
                      if label != -1]
        
        if not pred_labels:
            continue
            
        if row['gold'] in pred_labels:
            correct_label = row['gold']
        else:
            correct_label = pred_labels[0]  #
            
        y_true.append(row['gold'])
        y_pred.append(correct_label)
    
    #overall metrics
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    accuracy = accuracy_score(y_true, y_pred)
    
    class_report = classification_report(y_true, y_pred, output_dict=True)
    
    per_class_f1 = {}
    for class_label in sorted(set(y_true)):
        if str(class_label) in class_report:
            per_class_f1[class_label] = class_report[str(class_label)]['f1-score']
    
    return macro_f1, accuracy, per_class_f1


macro_f1, accuracy, per_class_f1 = evaluate_predictions(inference_df)

#overall metrics
print(f"Macro F1 Score: {macro_f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

#printing per-class F1 scores
print("\nPer-Class F1 Scores:")
print("\nClass\tF1 Score")
print("-" * 30)

distortion_names = {v: k for k, v in distortions.items()}

#printing per-class F1 scores with class names
for class_label, f1 in sorted(per_class_f1.items()):
    class_name = distortion_names.get(class_label, f"Class {class_label}")
    print(f"{class_name[:20]}\t{f1:.4f}")

Macro F1 Score: 0.1007
Accuracy: 0.1088

Per-Class F1 Scores:

Class	F1 Score
------------------------------
no distortion	0.1119
emotional reasoning	0.1143
overgeneralization	0.0667
mental filter	0.1200
should statements	0.0000
mind reading	0.0952
magnification/minimi	0.1333
personalization	0.1538
labeling	0.1111


In [48]:
#print where response == -1
inference_df[inference_df['Class_1_Label'] == -1].count()

Unnamed: 0        135
Prompt            135
Response          135
Class_1           135
Class_2           135
Class_1_Label     135
Class_2_Label     135
gold_dominant     135
gold_secondary    135
gold              135
dtype: int64